A股（至少是指数）是否有周内效应呢？就是特定周几盈利高一些，让我们来验证一下吧。

In [1]:
import pandas as pd
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
from datetime import datetime
import json
import numpy as np
import trdb2py

import plotly.express as px

isStaticImg = True

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [4]:
paramsaip = trdb2py.trading2_pb2.AIPParams(
    money=10000,
    type=trdb2py.trading2_pb2.AIPTT_MONTHDAY,
    day=1,
)    

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

buyup = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.30'],
)

buydown = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['down'],
    strVals=['ema.30'],
)

lststart = [1, 2, 3, 4, 5]
lstend = [2, 3, 4, 5, 1]
lsttitle = ['周一', '周二', '周三', '周四', '周五']
lstparams = []
assets = ['jrj.510310']
for day in range(0, 4):
    for i in range(0, 5):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[lststart[i]],
        )

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[trdb2py.nextWeekDay(lststart[i], day + 1)],
        )

        paramsbuy = trdb2py.trading2_pb2.BuyParams(
            perHandMoney=1,
        )

        paramssell = trdb2py.trading2_pb2.SellParams(
            perVolume=1,
        )
        
        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2Asset(assets[0]),
        )

        s0.buy.extend([buy0, buyup])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2Asset('jrj.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s0],
            title='{}{}up'.format(day, i),
        ))
        
        s1 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2Asset(assets[0]),         
        )

        s1.buy.extend([buy0, buydown])
        s1.sell.extend([sell0])
        s1.paramsBuy.CopyFrom(paramsbuy)
        s1.paramsSell.CopyFrom(paramssell)
        s1.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2Asset('jrj.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s1],
            title='{}{}down'.format(day, i),
        ))        
        
lstpnlaip = trdb2py.simTradings(trdb2cfg, lstparams)

# lstpnlaip

# # baseline    
# buy0 = trading2_pb2.CtrlCondition(
#     name='buyandhold',
# )

# paramsbuy = trading2_pb2.BuyParams(
#     perHandMoney=1,
# )

# paramsinit = trading2_pb2.InitParams(
#     money=10000,
# )

# pnl = libtrdb2.simTradingEx3(trdb2cfg, assets, 0, -1, [buy0], None, paramsbuy, None, paramsinit, None)
# lstpnlaip.append({'title': 'baseline', 'pnl': pnl})

dfaip = trdb2py.buildPNLReport(lstpnlaip)

In [5]:
dfaip[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

,title,maxDrawdown,maxDrawdownStart,maxDrawdownEnd,totalReturns,sharpe,annualizedReturns,annualizedVolatility,variance
36,33up,0.144835,2019-04-19,2020-04-24,3.274880,2.004850,0.300688,0.135017,0.458474
26,23up,0.111255,2013-10-21,2014-04-21,3.221435,2.190240,0.293624,0.120363,0.344667
16,13up,0.108013,2015-11-09,2017-02-03,2.779105,1.930423,0.235158,0.106276,0.237108
34,32up,0.194929,2015-11-09,2016-06-24,2.694133,1.320554,0.223927,0.146852,0.254420
28,24up,0.178435,2018-01-24,2019-01-14,2.605500,1.599818,0.212211,0.113895,0.245070
18,14up,0.128242,2018-01-23,2019-01-14,2.391166,1.543607,0.183881,0.099689,0.155493
8,04up,0.085687,2014-12-29,2015-01-23,2.213941,1.551346,0.160456,0.084092,0.108779
38,34up,0.274548,2017-11-22,2019-01-14,1.879073,0.645977,0.116194,0.133432,0.136422
24,22up,0.268510,2015-06-12,2016-06-24,1.866721,0.605727,0.114561,0.139603,0.085886
1,00down,0.177916,2015-06-30,2016-02-01,1.704633,0.866023,0.093137,0.072904,0.032206
